<a href="https://colab.research.google.com/github/greyhound101/Brain_tumour/blob/master/variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [38]:
import copy
from datetime import timedelta, datetime
import imageio
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing
import numpy as np
import os
from pathlib import Path
import pytest
import scipy.ndimage as ndimage
from scipy.ndimage.interpolation import zoom
from skimage import measure, morphology, segmentation
from time import time, sleep
from tqdm import trange, tqdm
from torchvision import transforms
import warnings
import scipy

In [39]:
import glob
image_paths=glob.glob('/content/gdrive/My Drive/ID*_segmented')
ids=[i.split('/')[-1].split('_')[0] for i in image_paths]

In [40]:
import pandas as pd
train = pd.read_csv('/content/gdrive/My Drive/lungs/train.csv') 
def get_tab(df):
    vector = [(df.Age.values[0] - 30) / 30] 
    
    if df.Sex.values[0] == 'male':
       vector.append(0)
    else:
       vector.append(1)
    
    if df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([0,0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([1,1])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0,1])
    else:
        vector.extend([1,0])
    return np.array(vector) 
A = {} 
TAB = {} 
P = [] 
for i, p in tqdm(enumerate(ids)):
    sub = train.loc[train.Patient == p, :] 
    fvc = sub.FVC.values
    weeks = sub.Weeks.values
    c = np.vstack([weeks, np.ones(len(weeks))]).T
    a, b = np.linalg.lstsq(c, fvc)[0]
    
    A[p] = a
    TAB[p] = get_tab(sub)
    P.append(p)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
174it [00:00, 675.62it/s]


In [70]:
from tensorflow.keras.utils import Sequence
class IGenerator(Sequence):
    def __init__(self, ids,a,image_paths, tab, batch_size=32):
        self.keys = ids
        self.a = a
        self.tab = tab
        self.batch_size = batch_size
        self.image_paths=image_paths
        self.train_data = {}
        
    def __len__(self):
        return 2000
    
    def __getitem__(self, idx):
        x= []
        a, tab = [], [] 
        keys = np.random.choice(self.keys, size = self.batch_size)
        for k in keys:
            #try:
                chosen_idx=self.keys.index(k)
                chosen_path=self.image_paths[chosen_idx]+'/image.npy'
                print(k==chosen_path.split('/')[-2].split('_')[0])
                img=np.load(chosen_path)
                x.append(img)
                a.append(self.a[k])
                tab.append(self.tab[k])
            
        x,a,tab = np.array(x), np.array(a), np.array(tab)
        x = np.expand_dims(x, axis=-1)
        return [x, tab] , [x,a]